In [1]:
import json

from api.db_models.weaviate_db import UserDocumentManager
from loading_utils import new_lines_to_list


In [2]:
import weaviate
client = weaviate.connect_to_local(
    host='localhost',
    port=8002,
    skip_init_checks=True
)

In [3]:
email = 'ceo@omegaintelligence.org'
company_name = 'ibis-research'

udm = UserDocumentManager(email, company_name, client)
SCHEMA = udm._schema

####
client.collections.delete(SCHEMA)
assert not client.collections.exists(SCHEMA)
####

In [4]:
s_prefix = 'IndustrySource/IBIS/'
p_prefix = 'IndustrySummary/IBIS/'
fnames = [
    'Furniture Stores in the US.pdf',
    'Household Furniture Manufacturing in the US.pdf',
    'IT Consulting in the US.pdf'
]
for fname in fnames:
    doc_path = s_prefix + fname
    summary_path = (p_prefix + fname).replace('.pdf', '.json')

    location = udm.upload_document_to_filestore(doc_path)

    with open(summary_path, 'r') as src:
        summary = json.load(src)
        summary = new_lines_to_list(summary)
        summary['location'] = location
        print(summary_path)
        udm.upload_document_summary(summary)

IndustrySummary/IBIS/Furniture Stores in the US.json
IndustrySummary/IBIS/Household Furniture Manufacturing in the US.json
IndustrySummary/IBIS/IT Consulting in the US.json


In [ ]:
result = udm.get_documents(
    title="IT Consulting Industry Report",
    # industry_name='Household Furniture Manufacturing'
)
print("Total", len(result.objects))
result.objects[0].properties

In [ ]:
assert client.collections.exists(SCHEMA)